# Calculate precipitation sum from GeoTiff radar images

Finnish Meteorological Institute is providing radar images as GeoTiff in S3 bucket. The data is free and openly available with CC4BY license.

This is a short and simple example how to calculate accumulated precipitation amount from GeoTiff radar images of precipitation rate (rr) with 5 minute interval. The example is ment to illustrate how to use the data with python.  

First we install necessary libraries. Rasterio (https://github.com/mapbox/rasterio) is used to fetch the data and pyproj (https://pypi.org/project/pyproj/) to calculate coordinate transformations.

In [1]:
!pip install rasterio[s3]

In [2]:
import rasterio as rio
from datetime import datetime, timedelta
from pyproj import Proj, Transformer

Small helper function to handle time steps.

In [3]:
def roundTime(dt, roundTo=300):
   """
   Round a datetime object to any time lapse in seconds
   
   dt      : datetime.datetime object
             DateTime to round
   roundTo : int
             Closest number of seconds to round to, default 1 minute. 
   
   Returns
   -------
   datetime.datetime object
            Modified datetime   
   """
   seconds = (dt.replace(tzinfo=None) - dt.min).seconds
   rounding = (seconds+roundTo/2) // roundTo * roundTo
   return dt + timedelta(0,rounding-seconds,-dt.microsecond)

Following function opens the geotiff image from S3 bucket and sample pixel value from requested coordinates. Note that images are in EPSG:3067 projection so we need to reproject the coordinates. 

Finally, pixel values are converted to millimeters. Conversions are documented here: https://en.ilmatieteenlaitos.fi/open-data-manual-radar-data.

In [4]:
def process_timestep(bucket, key, lat, lon, mode='rr', timestep=300):
    """ Process time step
    
    bucket   : str 
               S3 bucket name (where radar images exist)
    key      : str
               Radar image key in the bucket
    lat      : float
               Latitude of the point of interest
    lon      : float
               Longitude of the point of interest    
    mode     : str
               Defines how pixel values are converted to millimeters
    timestep : int
               Time step of radar images (used in pixel --> mm conversion)

    Returns
    -------
    float 
         Precipitation amount in millimeters
    """

    transformer = Transformer.from_crs("epsg:4326", "epsg:3067")
    x,y = transformer.transform(lat, lon)
    
    with rio.open('s3://{}/{}'.format(bucket, key)) as src:
        #print(src.profile)
        for val in src.sample([(x, y)]):
            if mode == 'rr':
                return val[0]*.01*timestep/3600
            else:
                raise Exception('mode not implemented')
        
    return 0

Last, we define a function to go through requested time range

In [5]:
def process_time_range(bucket, starttime, endtime, lat, lon, timestep, mode = 'rr', filename_start='', filename_end='_SUOMI500-rr.tif'):
    """ Process time range 
    bucket    : str 
                S3 bucket name (where radar images exist)
    starttime : str
                Start time of the time range in format %Y-%m-%d %H:%M:%S
    endtime   : str
                End time of the time range in format %Y-%m-%d %H:%M:%S                
    lat       : float
                Latitude of the point of interest
    lon       : float
                Longitude of the point of interest    
    timestep  : int
                Time step of radar images
    mode      : str
                Defines how pixel values are converted to millimeters
    filename_start : str
                     For example path of the image. Images are assumed to found with name 
                     filename_start+timestamp+filename_end.  
    filename_start : str
                 Images are assumed to found with name filename_start+timestamp+filename_end.  
                      
    Returns
    -------
    float 
         Precipitation amount in millimeters
    
    """
    
    start = roundTime(datetime.strptime(starttime, '%Y-%m-%d %H:%M:%S'))
    end = roundTime(datetime.strptime(endtime, '%Y-%m-%d %H:%M:%S'))    
    time_it = start
    
    prec_sum = 0
    while time_it <= end:
        key = '{}{}{}'.format(filename_start, time_it.strftime('%Y%m%d%H%M'), filename_end)
        prec_sum += process_timestep(bucket, key, lat, lon, timestep=timestep)

        if time_it.minute == 0: print('{:.2f}'.format(prec_sum), end='')
        else: print('.', end='')
            
        time_it +=  timedelta(seconds=timestep)

    print('')
    return prec_sum

And finally we run everything:

In [6]:
bucket = 'fmi-radar-opendata-sandbox'
starttime = '2020-07-10 00:00:00'
endtime = '2020-07-10 23:59:00'
lat = 63.1593
lon = 29.8346

prec_sum = process_time_range(bucket, starttime, endtime, lat, lon, 300)
print('Precipitation sum at {},{} (Koli, Finland) on 10th July 2020 is {:.2f} mm'.format(lat,lon,prec_sum))

0.00...........0.00...........0.03...........0.08...........0.09...........0.09...........0.09...........0.28...........0.30...........0.93...........2.14...........2.21...........2.51...........4.24...........4.63...........4.63...........5.13...........8.11...........8.11...........8.11...........8.11...........8.11...........8.11...........8.11...........8.11
Precipitation sum at 63.1593,29.8346 (Koli, Finland) on 10th July 2020 is 8.11 mm
